##### Note: before we start, it's good to remind the reader that this notebook is used to predict PsO: the skin disease called Psoraisis, based on phenotype data of 86499 patients.

In [25]:
import numpy as np
import pandas as pd

# I. Data Importation:

In [26]:
train_data = pd.read_csv('Ahmed_Data/train.csv')
test_data  = pd.read_csv('Ahmed_Data/test.csv')

In [27]:
print('the size if the training data set is ', train_data.shape)
print('the size if the test data set is ', train_data.shape)

the size if the training data set is  (73481, 78)
the size if the test data set is  (73481, 78)


In [28]:
train_data.head()

,SUBJID,BMI,ALCOHOL_STATUS,GENDER,REVISED_RPGEHRACE,SMOKING_STATUS,a1c_MEAN,a1c_UNDER,a1c_OVER,MEAN_bun,...,IBD,METABOLIC,MS,MYO_INFARC,PSORIASIS,PSOR_ARTH,RHEUM_ARTH,SCLERODERMA,SLE,VITILIGO
0,158108025613,23.0,1.0,2,other/uncertain,1.0,5.140000,0.0,0.000000,NaN,...,0,0,0,0,0,0,0,0,0,0
1,283455485374,32.0,1.0,2,white,1.0,6.100000,0.0,1.000000,15.571429,...,0,0,0,0,0,0,0,0,0,0
2,231997356863,37.0,0.0,2,white,1.0,6.183333,0.0,0.333333,15.000000,...,0,0,0,0,0,0,0,0,0,0
3,685384313765,31.0,1.0,1,hispanic,2.0,5.462069,0.0,0.000000,35.854167,...,0,0,0,1,0,0,0,0,0,0
4,229256381841,24.0,0.0,1,white,1.0,5.705000,0.0,0.050000,13.461538,...,0,0,0,0,0,0,0,0,0,0


# II. Further Data Preprocessing:

#### 1.Column "BMI"

In [29]:
# Let's find the number of the missing values in ALCOHOL_STATUS
print( 'There are', train_data[train_data.BMI.isnull()].shape[0], 'missing values in the BMI column in the training', 
       'data and', test_data[test_data.BMI.isnull()].shape[0], ' in the test data. We will replace all of them by '
       'the mean value of all the pateints')

There are 557 missing values in the BMI column in the training data and 103  in the test data. We will replace all of them by the mean value of all the pateints


In [30]:
# replacing the missing values in the BMI column by the mean value
train_data.BMI.fillna(train_data.BMI.mean(), inplace=True)
test_data.BMI.fillna(test_data.BMI.mean(), inplace=True)

#### 2. Column "ALCOHOL_STATUS"

In [31]:
train_data.ALCOHOL_STATUS.value_counts()

1.0    43313
0.0    27664
Name: ALCOHOL_STATUS, dtype: int64

In [32]:
# Let's find the number of the missing values in ALCOHOL_STATUS
print( 'the number of missing values in ALCOHOL_STATUS is ', train_data.shape[0] - train_data.ALCOHOL_STATUS.value_counts()[0] - train_data.ALCOHOL_STATUS.value_counts()[1])

the number of missing values in ALCOHOL_STATUS is  2504


#### NOTE: the SVM doesn't handle missing values. We will need to treat all of them before training the model.

In [33]:
# REPLACING MISSING VALUES
# SINCE 0=not a drinker, 1=drinker, we add 2= missing information 
train_data.ALCOHOL_STATUS.fillna(2, inplace=True)
test_data.ALCOHOL_STATUS.fillna(2, inplace=True)

#### 3. Column "REVISED_RPGEHRACE"

In [34]:
train_data.REVISED_RPGEHRACE.value_counts()

white              59502
asian               6066
hispanic            5041
black               2329
other/uncertain      543
Name: REVISED_RPGEHRACE, dtype: int64

In [35]:
# Since some ML algorithms only accept numeric input, we translate the strings into digits
train_data.REVISED_RPGEHRACE.replace(['white', 'asian','hispanic','black','other/uncertain'], [1,2,3,4,5], inplace=True)
test_data.REVISED_RPGEHRACE.replace(['white', 'asian','hispanic','black','other/uncertain'], [1,2,3,4,5], inplace=True)

#### 4. Column "SMOKING_STATUS"

In [36]:
# Let's find the number of the missing values in SMOKING_STATUS
print( 'There are', train_data.shape[0] - train_data.SMOKING_STATUS.value_counts()[1] - train_data.SMOKING_STATUS.value_counts()[2] - train_data.SMOKING_STATUS.value_counts()[3], 'missing values in SMOKING_STATUS ') 

There are 3109 missing values in SMOKING_STATUS 


In [37]:
# REPLACING MISSING VALUES
# Since 1=Never, 2=Former, 3=Current, we add 4=missing information
train_data.SMOKING_STATUS.fillna(4, inplace=True)
test_data.SMOKING_STATUS.fillna(4, inplace=True)

#### Columns 'a1c_MEAN', 'a1c_UNDER', 'a1c_OVER', 'MEAN_bun', 'UNDER_bun',... that have been created

#### NOTE: Those columns contain some missing values that are generated from the missing values in the test results for some patients

In [38]:
list_col = ['a1c_MEAN', 'a1c_UNDER', 'a1c_OVER', 'MEAN_bun',
       'UNDER_bun', 'OVER_bun', 'ccp_MEAN', 'ccp_UNDER', 'ccp_OVER',
       'MEAN_crea', 'UNDER_crea', 'OVER_crea', 'MEAN_CRP', 'OVER_CRP',
       'MEAN_ESR', 'OVER_ESR', 'MEAN_GLUC', 'UNDER_GLUC', 'OVER_GLUC',
       'MEAN_RHEUMATOID', 'UNDER_RHEUMATOID', 'OVER_RHEUMATOID',
       'v_d_MEAN', 'v_d_UNDER', 'v_d_OVER', 'Age', 'HDL_Normal',
       'HDL_Optimal', 'LDL_Optimal', 'LDL_Low_Risk', 'LDL_Normal_Risk',
       'LDL_High_Risk', 'LDL_Very_High_Risk', 'TRIGLYC_Optimal',
       'TRIGLYC_Low_Risk', 'TRIGLYC_Normal_Risk', 'TRIGLYC_High_Risk',
       'TRIGLYC_Very_High_Risk', 'ADALIMUMAB_RX', 'LEFLUNOMIDE_FILLS',
       'METHOTREXATE_FILLS', 'ADALIMUMAB_FILLS', 'USTEKINUMAB_FILLS',
       'CYCLOSPORINE_FILLS', 'ETANERCEPT_FILLS', 'INFLIXIMAB_FILLS',
       'ETANERCEPT_RX', 'ACITRETIN_RX', 'CYCLOSPORINE_RX',
       'ANTIRHEUMATIC_RX', 'ANTIRHEUMATIC_FILLS', 'LEFLUNOMIDE_RX',
       'USTEKINUMAB_RX', 'METHOTREXATE_RX', 'INFLIXIMAB_RX',
       'ACITRETIN_FILLS']
for i in list_col:
    mean_train = train_data[i].mean()
    mean_test = train_data[i].mean()
    train_data[i].fillna(mean_train, inplace=True)
    test_data[i].fillna(mean_test, inplace=True)
    

# III. First Model: k Nearest Neighbors

#### Note: There is no need here to have a validation set and a training set since we are using cross validation in the grid search function.

In [39]:
X_train =  train_data.drop(['PSORIASIS', 'SUBJID'], axis=1)
Y_train =  train_data.PSORIASIS

X_test =  test_data.drop(['PSORIASIS', 'SUBJID'], axis=1)
Y_test =  test_data.PSORIASIS

### 1. Tuning the model : finding the best parameters

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# we try values of K (nearest neighbors) from 1 to 5
k = np.arange(5)+1
parameters = {'n_neighbors' : k}
knn = KNeighborsClassifier()

# we use GridSearch wit a cross validation of 10 folds to find the best parameter K
knn_GridSearch = GridSearchCV(knn, parameters, cv=5)

# fit nearest neighbors to our training dataset
knn_GridSearch.fit(X_train,Y_train) 

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([1, 2, 3, 4, 5])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [18]:
# GridSearch give us the best parameter K:
knn_GridSearch.best_params_

{'n_neighbors': 4}

### 2. Training the model using the best parameters

In [106]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train,Y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')

### 3. Assessing the model

In [25]:
# Just for a simple evaluation, we could use the sklearn built-in method "score"
# Score method returns the mean accuracy on the given test data and labels
knn.score(X_test, Y_test)

0.9398519432449105

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection

seed = 0
kfold = model_selection.KFold(n_splits=10, random_state=seed)
knn = KNeighborsClassifier(n_neighbors=4)

#### NOTE: In this project, we will use diffrent classification evaluation metrics: 

##### 1. Classification Accuracy

Classification accuracy is the ratio (number of correct predictions / all predictions made).

Good to know about the metric:
- The most common evaluation metric for classification problems.
- Only suitable when there are an equal number of observations in each class (which is not the case in our unbalanced dataset) 
- Only suitable when prediction errors are equally important, which is often not the case here: False Negatives are clearly worse than False Positives in our situation.

In [41]:
## Classification Accuracy
scoring = 'accuracy'
results = model_selection.cross_val_score(knn, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

- Accuracy: 0.938 (0.002)


##### 2. Logarithmic Loss

Logarithmic loss (or logloss) is a performance metric for evaluating the predictions of probabilities of membership to a given class.

Good to know about the metric:
- The scalar probability between 0 and 1 can be seen as a measure of confidence for a prediction by an algorithm. Predictions that are correct or incorrect are rewarded or punished proportionally to the confidence of the prediction.


In [42]:
## Logarithmic Loss
scoring = 'neg_log_loss'
results = model_selection.cross_val_score(knn, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Logloss: %.3f (%.3f)" % (results.mean(), results.std()))

- Logloss: -1.657 (0.069)


##### 3. Area under ROC Curve

The AUC represents a model’s ability to discriminate between positive and negative classes. An area of 1.0 represents a model that made all predictions perfectly. An area of 0.5 represents a model as good as random. 

In [43]:
## Area Under ROC Curve
scoring = 'roc_auc'
results = model_selection.cross_val_score(knn, X_train, Y_train, cv=kfold, scoring=scoring)
print("- AUC: %.3f (%.3f)" % (results.mean(), results.std()))

- AUC: 0.539 (0.010)


##### 4. Confusion Matrix (or error matrix)

It is a specific table layout that allows visualization of the performance of an algorithm. Each row of the matrix represents the instances in a predicted class. It is a special kind of contingency table, with two dimensions ("actual" and "predicted").

In [44]:
## Cross Validation Classification Confusion Matrix

from sklearn.metrics import confusion_matrix

knn.fit(X_train, Y_train)
predicted = knn.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print("- Confusion Matrix: ")
print(matrix)

- Confusion Matrix: 
[[12166    19]
 [  761    22]]


# IV. Second Model: Naive Bayes

### 1. Gaussian Naive Bayes

#### 1.1 Training the model

In [45]:
# training the model
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(X_train, Y_train)

GaussianNB(priors=None)

#### 1.2 Evaluating the model

In [46]:
# number of mislabeld data in test set
y_pred = GNB.predict(X_test)

In [47]:
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0],(Y_test != y_pred).sum()))

Number of mislabeled points out of a total 12968 points : 651


In [48]:
# Returns the mean accuracy on the given test data and labels
GNB.score(X_test, Y_test, sample_weight=None)

0.94979950647748301

In [49]:
from sklearn.metrics import log_loss
print(log_loss(Y_test, GNB.predict(X_test)))

1.7338690467


In [103]:
## Logarithmic Loss
from sklearn import model_selection

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
GNB = GaussianNB()


## Classification Accuracy
scoring = 'accuracy'
results = model_selection.cross_val_score(GNB, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

## Logarithmic Loss
scoring = 'neg_log_loss'
results = model_selection.cross_val_score(GNB, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Logloss: %.3f (%.3f)" % (results.mean(), results.std()))

## Area Under ROC Curve
scoring = 'roc_auc'
results = model_selection.cross_val_score(GNB, X_train, Y_train, cv=kfold, scoring=scoring)
print("- AUC: %.3f (%.3f)" % (results.mean(), results.std()))

## Cross Validation Classification Confusion Matrix

from sklearn.metrics import confusion_matrix

GNB.fit(X_train, Y_train)
predicted = GNB.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print("- Confusion Matrix: ")
print(matrix)

- Accuracy: 0.947 (0.002)
- Logloss: -0.223 (0.009)
- AUC: 0.678 (0.013)
- Confusion Matrix: 
[[12099    86]
 [  581   202]]


### 2. Multinomial Naive Bayes

#### 2.1 Tuning the model : finding the best parameters

In [60]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# we try values of K (nearest neighbors) from 1 to 5
alpha = np.arange(100)+1
parameters = {'alpha' : alpha}
MNB = MultinomialNB()

# we use GridSearch wit a cross validation of 10 folds to find the best parameter K
MNB_GridSearch = GridSearchCV(MNB, parameters, cv=10)

# fit nearest neighbors to our training dataset
MNB_GridSearch.fit(X_train,Y_train) 

GridSearchCV(cv=10, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1,   2, ...,  99, 100])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [61]:
MNB_GridSearch.best_params_

{'alpha': 58}

#### 2.2 Training the model using the best parameters

In [64]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB(alpha=58.0, fit_prior=True, class_prior=None)
MNB.fit(X_train, Y_train)

MultinomialNB(alpha=58.0, class_prior=None, fit_prior=True)

#### 2.3 Assessing the model

In [65]:
# number of mislabeld data in test set
y_pred = MNB.predict(X_test)

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0],(Y_test != y_pred).sum()))

Number of mislabeled points out of a total 12968 points : 667


In [66]:
# Returns the mean accuracy on the given test data and labels
MNB.score(X_test, Y_test, sample_weight=None)

0.94856570018507091

In [69]:
from sklearn.metrics import log_loss
print(log_loss(Y_test, MNB.predict(X_test)))

1.77648308303


In [104]:
## Logarithmic Loss
from sklearn import model_selection

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
MNB = MultinomialNB(alpha=58.0, fit_prior=True, class_prior=None)


## Classification Accuracy
scoring = 'accuracy'
results = model_selection.cross_val_score(MNB, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

## Logarithmic Loss
scoring = 'neg_log_loss'
results = model_selection.cross_val_score(MNB, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Logloss: %.3f (%.3f)" % (results.mean(), results.std()))

## Area Under ROC Curve
scoring = 'roc_auc'
results = model_selection.cross_val_score(MNB, X_train, Y_train, cv=kfold, scoring=scoring)
print("- AUC: %.3f (%.3f)" % (results.mean(), results.std()))

## Cross Validation Classification Confusion Matrix

from sklearn.metrics import confusion_matrix

MNB.fit(X_train, Y_train)
predicted = MNB.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print("- Confusion Matrix: ")
print(matrix)

- Accuracy: 0.947 (0.002)
- Logloss: -0.223 (0.009)
- AUC: 0.678 (0.013)
- Confusion Matrix: 
[[12099    86]
 [  581   202]]


# V. Third Model : Support Vector Machine

#### Note: Here we use Support Vector Machine” (SVM) as a supervised machine learning algorithm in a purpose of classification

In [51]:
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data.drop(['PSORIASIS', 'SUBJID'], axis=1),
                                                      train_data.PSORIASIS,
                                                      test_size=0.33, random_state=42)

X_test =  test_data.drop(['PSORIASIS', 'SUBJID'], axis=1)
Y_test =  test_data.PSORIASIS

#### Note: One of the disadvatages of SVM is that it is computationally expensive, thus runs slow. It is then not time-efficient to run a grid search in order to find the best parameters. As a result, I will run diffrent 

### 1. Training the model with diffrent parameters

In [38]:
# train an SVM classification model with C=1 and kernel = rbf "radial basis function"

svm_1 = sklearn.svm.SVC(C=1, kernel='rbf')
svm_1.fit(X_train,Y_train)
svm_1.score(X_valid, Y_valid, sample_weight=None)

0.93777062971668934

In [39]:
# train an SVM classification model with C=10 and kernel = rbf "radial basis function"

svm_2 = sklearn.svm.SVC(C=10, kernel='rbf')
svm_2.fit(X_train,Y_train)
svm_2.score(X_valid, Y_valid, sample_weight=None)

0.93146109117901765

In [40]:
# train an SVM classification model with C=0.1 and kernel = rbf "radial basis function"

svm_3 = sklearn.svm.SVC(C=0.1, kernel='rbf')
svm_3.fit(X_train,Y_train)
svm_3.score(X_valid, Y_valid, sample_weight=None)

0.93789434615860445

In [41]:
# train an SVM classification model with C=0.01 and kernel = rbf "radial basis function"

svm_4 = sklearn.svm.SVC(C=0.01, kernel='rbf')
svm_4.fit(X_train,Y_train)
svm_4.score(X_valid, Y_valid, sample_weight=None)

0.93789434615860445

In [42]:
# train an SVM classification model with C=100 and kernel = rbf "radial basis function"

svm_5 = sklearn.svm.SVC(C=100, kernel='rbf')
svm_5.fit(X_train,Y_train)
svm_5.score(X_valid, Y_valid, sample_weight=None)

0.91694502866097571

### 2. Evaluation the model:

In [52]:
from sklearn import model_selection

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
svm = sklearn.svm.SVC(C=0.1, kernel='rbf')

In [ ]:
## Classification Accuracy
scoring = 'accuracy'
results = model_selection.cross_val_score(svm, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

- Accuracy: 0.939 (0.002)


In [ ]:
## Logarithmic Loss
scoring = 'neg_log_loss'
results = model_selection.cross_val_score(svm, X_train, Y_train, cv=kfold, scoring=scoring)
print("- Logloss: %.3f (%.3f)" % (results.mean(), results.std()))

In [ ]:
## Area Under ROC Curve
scoring = 'roc_auc'
results = model_selection.cross_val_score(svm, X_train, Y_train, cv=kfold, scoring=scoring)
print("- AUC: %.3f (%.3f)" % (results.mean(), results.std()))

In [ ]:
## Cross Validation Classification Confusion Matrix
from sklearn.metrics import confusion_matrix

svm.fit(X_train, Y_train)
predicted = svm.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print("- Confusion Matrix: ")
print(matrix)